In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/MyDrive/PROJECTS/gan')

In [ ]:
RESULT_DIR = '/content/gdrive/MyDrive/PROJECTS/gan/experiments/celeb_a/results'

In [ ]:
!unzip /content/gdrive/MyDrive/DATASETS/Celeb_A/celeba-dataset.zip
DATA_DIR = '/content/img_align_celeba/img_align_celeba'

In [ ]:
import tensorflow as tf
import os

dataset = tf.keras.utils.image_dataset_from_directory(
    directory=DATA_DIR,
    labels=len(os.listdir(DATA_DIR)) * [1],
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=256,
    image_size=(64,64),
    shuffle=True,
    seed=42,
).map(
    lambda x,y: (x/(255/2)-1, y)
)

virtual_batch, _ = dataset.rebatch(2048).as_numpy_iterator().next()

In [ ]:
from experiments.celeb_a.architecture import generator, discriminator
from experiments.utils import VisualizeGeneratedFakesCallback
from gan import GenerativeAdversarialNetwork

z_shape=generator.input_shape[1:]
x_shape=discriminator.input_shape[1:]

gan = GenerativeAdversarialNetwork(generator, discriminator, z_shape, x_shape, k=1,
                                  label_smooting=(0, 0.9),
                                   historical_averaging=True,
                                   feature_matching_idx=1,
                                   virtual_normalization_batch=virtual_batch
                                   )

gan.summary()

gan.compile(optimizer_discriminator=tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
            optimizer_generator=tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5))
callbacks=[VisualizeGeneratedFakesCallback(z_distribution=gan.z_distribution, save_dir=RESULT_DIR)]
gan.fit(x=dataset, callbacks=callbacks, epochs=100)